## Implementation of interior point method for Linear Programs

### Althoght not explicitly asked in the assignment, we decided to implement the interior point method as well

### However, the implementation currently stands unsuccessful, due to divergence of solution (tested for a small toy problem)

In [31]:
import numpy as np

In [32]:
# define function to find initial iterate
# ----Section 14.2 
# ------Subsection: Starting Point

def O(m,n):
    return np.zeros((m,n))

def init(A, b, c):
    AAt = np.matmul(A, A.T)
    AAt_inv = np.linalg.inv(AAt)
    
    x_bar = np.matmul( np.matmul(A.T, AAt_inv), b )
    lam_bar = np.matmul( np.matmul(AAt_inv, A), c )
    s_bar = c - np.matmul(A.T, lam_bar)
    
    delx = max([-1.5*np.min(x_bar), 0])
    dels = max([-1.5*np.min(s_bar), 0])
    
    x_cap = x_bar + delx
    s_cap = s_bar + dels
    
    delx_cap = 0.5*( np.dot(x_cap, s_cap)/np.sum(s_cap) )
    dels_cap = 0.5*( np.dot(x_cap, s_cap)/np.sum(x_cap) )
    
    x_o = x_cap + delx_cap
    lam_o = lam_bar
    s_o = s_cap + dels_cap
    
    return x_o, lam_o, s_o

In [33]:
# get del_x_aff, del_lam_aff, del_s_aff
def solve_For_Newton_Dir(A, x_k, lam_k, s_k, b, c):
    m,n = A.shape
#     print("A.shape", A.shape)
    Coef_mat = np.block([   [O(n,n),       A.T,    np.eye(n)],
                            [A,            O(m,m), O(m,n)   ],
                            [np.diag(s_k), O(n,m), np.diag(x_k)]])
    
    rc = np.matmul(A.T, lam_k) + s_k - c
    
    rb = np.matmul(A, x_k) - b

    rxs= np.multiply(x_k, s_k)
    
#     print(rc.shape, rb.shape, rxs.shape)
    RHS = np.concatenate((-rc, -rb, -rxs))
    
    soln = np.linalg.solve(Coef_mat, RHS)
#     print(soln.shape)
    
    del_x_aff, del_lam_aff, del_s_aff =(soln[0:n], soln[n:n+m], soln[n+m: 2*n + m])
    
    return del_x_aff, del_lam_aff, del_s_aff

In [34]:
# calculate alpha_aff_prim, alpha_aff_dual, mu_aff
def calc_Alpha_affs(del_x_aff, x_k, del_s_aff, s_k):

    list_ax = [float('inf')]
    list_as = [float('inf')]
    n = len(del_x_aff)
    
    for i in range(n):
        if del_x_aff[i] < 0:
            list_ax.append(-x_k[i]/del_x_aff[i])
            
        if del_s_aff[i] < 0:
            list_as.append(-s_k[i]/del_s_aff[i])
                           
    list_ax_min = min(list_ax)
    list_as_min = min(list_as)
            
    alpha_aff_prim = min(1, list_ax_min)
    alpha_aff_dual = min(1, list_as_min)
    
    mu_aff = np.dot( (x_k + alpha_aff_prim*del_x_aff) , (s_k + alpha_aff_dual*s_k) ) / n
    mu =  np.dot(x_k, s_k)/n
    
    sigma = (mu_aff/mu)**3
    
    return alpha_aff_prim, alpha_aff_dual, mu, mu_aff, sigma

In [35]:
# get del_x, del_lam, del_s
def solve_For_Search_Dir(A, x_k, lam_k, s_k, b, c, sigma, mu, del_x_aff, del_s_aff):
    m,n = A.shape
    Coef_mat = np.block([ [O(n,n),       A.T,    np.eye(n)],
                            [A,            O(m,m), O(m,n)   ],
                            [np.diag(s_k), O(n,m), np.diag(x_k)]])
    
    rc = np.matmul(A.T, lam_k) + s_k - c
    rb = np.matmul(A, x_k) - b
    rxs= - np.multiply(x_k, s_k) - np.multiply(del_x_aff, del_s_aff) + (sigma*mu) 
    
#     print(rc.shape, rb.shape, rxs.shape)
    RHS = np.concatenate((-rc, -rb, rxs))
    
    soln = np.linalg.solve(Coef_mat, RHS)
#     print(soln.shape)
    
    del_x, del_lam, del_s = (soln[0:n], soln[n:n+m], soln[n+m: 2*n + m])
    
    return del_x, del_lam, del_s

In [36]:
# calc alha_prim and alpha_dual
def calc_Alphas(x_k, del_x, s_k, del_s, n_k):
    
    list_ax = [float('inf')]
    list_as = [float('inf')]
    n = len(del_x)
    
    for i in range(n):
        if del_x[i] < 0:
            list_ax.append(-x_k[i]/del_x[i])
            
        if del_s[i] < 0:
            list_as.append(-s_k[i]/del_s[i])
                           
    alpha_prim_max = min(list_ax)
    alpha_dual_max = min(list_as)
            
    alpha_prim = min(1, n_k*alpha_prim_max)
    alpha_dual = min(1, n_k*alpha_dual_max)
    
    return alpha_prim, alpha_dual

In [37]:
def next_iterate(x_k, lam_k, s_k, alpha_prim, alpha_dual, del_x, del_lam, del_s):
    x_next = x_k + alpha_prim*del_x
    lam_next = lam_k + alpha_dual*del_lam
    s_next = s_k + alpha_dual*del_s
    return x_next, lam_next, s_next

In [40]:
test_A = np.asarray([[1,1,1,0],[2,0.5,0,1]])
test_b = np.asarray([5,8])
test_c = np.asarray([-4,-2,0,0])


# main loop
x_k, lam_k, s_k = init(test_A, test_b, test_c)
print("init guess")
print(x_k, lam_k, s_k)
print('duality measure:')
nsteps = 10
for i in range(nsteps):
   
    del_x_aff, del_lam_aff, del_s_aff = solve_For_Newton_Dir(test_A, x_k, lam_k, s_k, test_b, test_c)
    
    alpha_aff_prim, alpha_aff_dual, mu, mu_aff, sigma = calc_Alpha_affs(del_x_aff, x_k, del_s_aff, s_k)
    
    del_x, del_lam, del_s = solve_For_Search_Dir(test_A, x_k, lam_k, s_k, test_b, test_c, sigma, mu, del_x_aff, del_s_aff)
    
    n_k = 0.9 + 0.1*(i/nsteps)
    alpha_prim, alpha_dual = calc_Alphas(x_k, del_x, s_k, del_s, n_k)
    
    x_k, lam_k, s_k = next_iterate(x_k, lam_k, s_k, alpha_prim, alpha_dual, del_x, del_lam, del_s)
    print(mu)
    if mu <10e-4:
        print("Solution found")
        break
      

init guess
[3.63345865 1.81766917 1.21240602 1.76503759] [-0.94736842 -1.26315789] [0.65789474 0.76315789 2.13157895 2.44736842]
duality measure
2.670409576573012
2.770168473584138
5.4829634904271956
20.820396325989673
131.13509142992388
1031.778001802609
8217.31052709724
65721.4612675084
525734.1797539616
4205854.490658309
